<img src="images/scilife_logo.png" width="400">
<img src="images/essence_logo.png" width="300">

# SciLifeLab Workshop - Hands-on Section: LangGraph "Hello World"



Welcome to this hands‑on lab session on building AI Agents with **LangGraph**! 

LangGraph is a low‑level orchestration framework for constructing stateful AI workflows using graphs. LangGraph provides several key benefits for agentic AI applications, including durable execution, support for human‑in‑the‑loop workflows, comprehensive memory (both short‑ and long‑term), built‑in debugging, and production‑ready deployment features.

## Learning Objectives
By the end of this workshop, you will:
- Understand core concepts of LangGraph (tools, nodes, edges, state, and memory).
- Create and integrate your own tools for an AI agent.
- Build a ReAct‑style agent using an LLM and custom tools.
- Implement agent memory to maintain conversational context.
- Compare custom agents with prebuilt LangGraph agents.
- Explore extension tasks such as custom graph, structured output, and prompts template

## Workshop Outline 
- **Part 1**: Setup & imports
- **Part 2**: Understanding and creating tools
- **Part 3**: Defining the state
- **Part 4**: Building the agent graph
- **Part 5**: Testing the agent
- **Part 6**: Adding short‑term memory
- **Part 7**: Exploring prebuilt agents
- **Part 8 (optional)**: Extension exercises

## Instructions for Participants

**Throughout this lab, look for `TODO` comments and `...` placeholders in the code cells. These indicate where you need to add your implementation.** 

Your task is to:
1. Replace `...` placeholders with appropriate code
2. Follow the instructions in `TODO` comments 
3. Refer to the exercise descriptions and API references provided
4. Use the `test cell` right below each `TODO` cell to validate your solutions

**Tip:** Each exercise builds on the previous one, so complete them in order!

---

## Part 1 – Setup

In this first step, we'll import the necessary dependencies and load any environment variables. Make sure your API keys (e.g., OpenAI) are stored in a `.env` file in the same directory.

### Exercise 1.1 – Import dependencies

In [ ]:
# Import any packages you need here
import json
from dotenv import load_dotenv

load_dotenv(override=True)

---

## Part 2 – Understanding and Creating Tools

In LangGraph, **tools** are Python functions that extend your agent's capabilities beyond text generation. They can call external APIs or perform computations, and are annotated with the `@tool` decorator from LangChain. A tool takes typed inputs and returns a string; the LLM can decide when to call a tool.

When designing tools for agents, there are two important considerations: 
- **Input - Output**: It is similar to when designing a Python function, but the input is now handled by generated content from LLM. Output should be parsed to feed meaningful context for the agent context 

- **Description**: The `@tool` decorator requires docstrings from a Python function and will use it as the tool's description. These descriptions will feed the LLM system prompts, making it aware of these existing tools. 

### About the task

In this exercise, we will develop three tools useful for the drug discovery domain.

**1. Calculator:** a tool to execute mathematical expressions

**2. LitSearch:** a tool to perform semantic search on all publications available on PubMed

**3. GetDrugInfo:** a tool to retrieve drug information given the drug name

### Exercise 2.1 – Create a Calculator Tool

Write a tool that evaluates simple arithmetic expressions (e.g. `'2 + 3 * 4'`). 

Instruction: 
- Use Python's built‑in `eval` to perform the calculation.
- Your tool should accept a `string` input called `expression` and return a `string` result.

In [107]:
from langchain.tools import tool

# TODO: establish calculator tool
@tool
def calculator(expression: str) -> str:
    """Evaluate a simple arithmetic expression (e.g. '2 + 3 * 4').
    
    Args:
        expression (str): A string containing a valid arithmetic expression (e.g., "2 + 3 * 4").
    Returns:
        result (str): The result of evaluating the expression, converted to a string."""
    return "..."

In [ ]:
# Test for Exercise 2.1
try:
    # ensure calculator returns correct result for simple expressions
    assert callable(calculator), "calculator must be callable"
    result1 = calculator("2 + 3 * 4")
    assert isinstance(result1, str), "calculator should return a string"
    assert result1.strip() == "14", f"Expected '14', got {result1}"
    # check error handling
    err_result = calculator("2 +")
    assert isinstance(err_result, str) and ("error" in err_result.lower() or "invalid" in err_result.lower()), "Calculator should return an error string on invalid input"
    print("Calculator tool tests passed!")
except Exception as e:
    raise AssertionError(f"Calculator tool tests failed: {e}")


### Exercise 2.2 – Create a Literature Search Tool

Use the **LitSense** API to perform semantic search of PubMed articles. Your tool should accept a `query` string and an optional `limit` integer. 

Below is the API reference of LitSense Wrapper.

<img src="./images/LitSense_API.png" width="800">


### **References**
LitSense: https://academic.oup.com/nar/article/53/W1/W361/8133630

Github: https://github.com/DinhLongHuynh/LitSense_Wrapper

In [112]:
from langchain.tools import tool
from utils.litsense import LitSense_API

@tool
def lit_search(query: str, limit: int = 5) -> str:
    """Retrieve information from PubMed using a semantic search via the LitSense API.\n\n
    Args:
        query: The research question or topic to search for in PubMed literature.
        limit: Maximum number of results to return (default is 5).
    Returns:
        result (str): A formatted string containing semantically relevant passages from PubMed articles, including PMID and content only.
    """

    # TODO: establish engine and get the results given query
    engine = ...
    results = ...

    # Parse result into prefered format
    result_str = ""
    for i, result in enumerate(results):
        result_str += (
                f"\n--- Passage #{i+1} ---\n"
                f"PMID: {result.pmid}\n"
                f"Content: {result.text}\n"
            )
    return result_str

In [ ]:
# Test for Exercise 2.2 
try:
    class _DummyResult:
        def __init__(self, pmid, text):
            self.pmid = pmid
            self.text = text

    class _DummyEngine:
        def __init__(self):
            self.retrieve_called = False
            self.last_args = None
        def retrieve(self, query, limit=5):
            self.retrieve_called = True
            self.last_args = (query, limit)
            return [_DummyResult('PMID1', 'Sample text 1'), _DummyResult('PMID2', 'Sample text 2')]

    target_callable = getattr(lit_search, 'func', lit_search)
    func_globals = getattr(target_callable, '__globals__', None)
    assert func_globals is not None, 'lit_search callable must expose globals for patching'

    import utils.litsense as litsense_module
    has_global_cls = 'LitSense_API' in func_globals
    original_cls_global = func_globals.get('LitSense_API')
    original_cls_module = getattr(litsense_module, 'LitSense_API', None)
    dummy_engine_instance = _DummyEngine()
    res = None

    func_globals['LitSense_API'] = lambda: dummy_engine_instance
    if original_cls_module is not None:
        litsense_module.LitSense_API = lambda: dummy_engine_instance

    try:
        call_kwargs = {'query': 'test query', 'limit': 2}
        if hasattr(lit_search, 'invoke'):
            res = lit_search.invoke(call_kwargs)
        elif hasattr(lit_search, 'run'):
            try:
                res = lit_search.run(call_kwargs)
            except TypeError:
                res = lit_search.run('test query')
        else:
            res = lit_search(**call_kwargs)
    finally:
        if has_global_cls:
            func_globals['LitSense_API'] = original_cls_global
        else:
            func_globals.pop('LitSense_API', None)
        if original_cls_module is not None:
            litsense_module.LitSense_API = original_cls_module

    assert dummy_engine_instance.retrieve_called, 'lit_search should call LitSense_API.retrieve'
    assert dummy_engine_instance.last_args == ('test query', 2), 'lit_search should pass query and limit to retrieve'
    assert isinstance(res, str), 'lit_search should return a formatted string'
    assert 'PMID:' in res and 'Content:' in res, 'Result should include PMID and Content fields'
    print('Literature search tool tests passed!')
except Exception as e:
    raise AssertionError(f'Literature search tool tests failed: {e}')

### Exercise 2.3 – Create a Drug Information Tool

Build a tool that queries the **ChEMBL** database for comprehensive drug information. Use the `chembl_webresource_client` to search for molecules by name or synonym. If you find multiple candidates, select the first one. Return a formatted string containing the molecule's name, mechanism of action, and therapeutic indications.

Due to the complexity of the API endpoint, we decided to give you the full script for this tool so you could focus on building an agent with LangGraph.

In [85]:
from langchain.tools import tool
from chembl_webresource_client.new_client import new_client

@tool
def get_drug_info(drug_name: str) -> str:
    """Retrieve comprehensive drug information using the ChEMBL database (supports synonyms).
    
    Args:
        drug_name (str): the drug name of interest
    Returns:
        result (str): A formatted string containing drug information from ChEMBL.
    """
    
    try:
        # Search for the molecule - get only first result
        search_results = new_client.molecule.search(drug_name)
        
        try:
            candidate = next(iter(search_results))
        except StopIteration:
            return f"No information found for '{drug_name}'"

        chembl_id = candidate['molecule_chembl_id']
        
        # Get molecule details (single API call)
        details = new_client.molecule.get(chembl_id)

        # Build base result string
        pref_name = details.get('pref_name') or candidate.get('pref_name') or drug_name
        result = f"**{pref_name}** (ChEMBL ID: {chembl_id})\n"
        result += f"Type: {details.get('molecule_type', 'Not specified')}\n"

        # Add molecular formula if available
        mol_props = details.get('molecule_properties') or {}
        if mol_props.get('molecular_formula'):
            result += f"Molecular Formula: {mol_props['molecular_formula']}\n"

        # Add synonyms - already in details
        synonyms = details.get('molecule_synonyms') or []
        if synonyms:
            result += "\n**Synonyms/Trade names:**\n"
            seen = set()
            count = 0
            for syn in synonyms:
                if count >= 8:  # Limit to 8 synonyms
                    break
                name = syn.get('molecule_synonym') or syn.get('synonyms')
                if name and name.lower() != pref_name.lower() and name not in seen:
                    seen.add(name)
                    result += f"- {name}\n"
                    count += 1

        # Fetch mechanisms - limited to 3
        try:
            mechs = new_client.mechanism.filter(
                molecule_chembl_id=chembl_id
            ).only(['mechanism_of_action'])[:3]
            mechanism_list = list(mechs)
            
            if mechanism_list:
                result += "\n**Mechanism of Action:**\n"
                seen_moa = set()
                for mech in mechanism_list:
                    moa = mech.get('mechanism_of_action')
                    if moa and moa not in seen_moa:
                        seen_moa.add(moa)
                        result += f"- {moa}\n"
        except:
            pass

        # Fetch indications - limited to 6
        try:
            inds = new_client.drug_indication.filter(
                molecule_chembl_id=chembl_id
            ).only(['efo_term', 'mesh_heading', 'max_phase_for_ind'])[:6]
            indication_list = list(inds)
            
            if indication_list:
                result += "\n**Indications:**\n"
                seen_ind = set()
                for ind in indication_list:
                    indication_name = ind.get('efo_term') or ind.get('mesh_heading')
                    if indication_name and indication_name not in seen_ind:
                        seen_ind.add(indication_name)
                        max_phase = ind.get('max_phase_for_ind')
                        phase_info = f" (Phase {max_phase})" if max_phase else ""
                        result += f"- {indication_name}{phase_info}\n"
            else:
                result += "\nNo indication data available.\n"
        except:
            result += "\nNo indication data available.\n"

        # Fetch activities - limited to 30
        try:
            acts = new_client.activity.filter(
                molecule_chembl_id=chembl_id
            ).only(['standard_type'])[:30]
            activity_list = list(acts)
            
            if activity_list:
                result += f"\n**Bioactivity Data:** {len(activity_list)}+ records\n"
                activity_types = {}
                for act in activity_list:
                    act_type = act.get('standard_type')
                    if act_type:
                        activity_types[act_type] = activity_types.get(act_type, 0) + 1
                if activity_types:
                    top_types = sorted(activity_types.items(), key=lambda x: x[1], reverse=True)[:3]
                    result += "Top types: " + ", ".join([f"{t[0]} ({t[1]})" for t in top_types]) + "\n"
        except:
            pass

        return result

    except Exception as e:
        return f"Error querying ChEMBL database for '{drug_name}': {str(e)}"

### Exercise 2.4 – Create a Tools List

Collect all of your tool functions into a single list called `TOOLS`. This list will be passed to the LLM so it is aware of the available tools.

In [86]:
# TODO: create a list of tools
TOOLS = [...]

In [ ]:
# Test for Exercise 2.4 
try:
    assert isinstance(TOOLS, list), "TOOLS should be a list"
    # Extract names from tool objects; decorated tools expose a 'name' attribute
    tool_names = set()
    for t in TOOLS:
        name = getattr(t, 'name', None)
        if name is None:
            # Fallback to function name
            name = getattr(t, '__name__', None)
        tool_names.add(name)
    expected = {'calculator', 'lit_search', 'get_drug_info'}
    assert expected.issubset(tool_names), f"TOOLS should contain calculator, lit_search, and get_drug_info. Found {tool_names}"
    print("TOOLS list tests passed!")
except Exception as e:
    raise AssertionError(f"TOOLS list tests failed: {e}")


---

## Part 3 – Understanding LangGraph State

For agents, the state typically contains a list of messages that **grows over** the conversation. 

When creating a state schema, you use:

- `TypedDict` to describe the keys.
- `Annotated` with updating functions to specify how values should be updated. 
- `add_messages` is one updating function. It appends new messages rather than overwriting them.



### Exercise 3.1 – Define Chat State

Instruction: 
- Define a `ChatState` class (subclassing `TypedDict`) with a single key `messages`. 
- Use the `add_messages` updating function so that new messages are appended to the state.

Below is the API reference of `ChatState`.

<img src="images/ChatState.png" width="800">

In [ ]:
from typing_extensions import TypedDict
from typing import Annotated, List, Dict
from langgraph.graph.message import add_messages

# TODO: define graph state
class ChatState(...):
    """The state schema for our LangGraph. It contains only a list of messages.

    Messages are appended via the `add_messages` to preserve the full conversation history."""
    messages: ...

In [ ]:
# Test for Exercise 3.1
try:
    from typing import get_origin
    assert isinstance(ChatState, type), "ChatState should be a class"
    assert 'messages' in getattr(ChatState, '__annotations__', {}), "ChatState should define a 'messages' field"
    annotation = ChatState.__annotations__['messages']
    # Check that the annotation uses Annotated with add_messages metadata
    # For Annotated types, __metadata__ holds metadata
    has_metadata = hasattr(annotation, '__metadata__')
    assert has_metadata, "ChatState.messages should be Annotated to include add_messages metadata"
    # Verify the metadata includes add_messages
    from langgraph.graph.message import add_messages
    metadata = annotation.__metadata__
    assert add_messages in metadata, "ChatState.messages Annotated metadata should include add_messages"
    print("ChatState definition tests passed!")
except Exception as e:
    raise AssertionError(f"ChatState definition tests failed: {e}")

---

## Part 4 – Building the Agent Graph

This is the graph we are going to create: 

<img src="images/react_agent.png" width="300">

### Exercise 4.1 – Initialize Components

Instruction: 
- Initialise a `StateGraph` instance using your `ChatState`.
- Initialise a chat model with `ChatOpenAI(model='gpt-4o', temperature=0)`.
- Bind your tools to the LLM using `bind_tools()` so that the LLMS knows tools' schemas and how to construct tool calls
- Create a `ToolNode` from your tools list. This class serves as a wrapper for the tools and should implement all necessary methods, including parsing the LLM's output, managing the tool schemas, and handling the output from the tools.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI

# TODO: initialize graph builder using ChatState
graph_builder = ...

# TODO: Initialise the chat model. Ensure your OpenAI API key is set in the .env file
llm = ChatOpenAI(model='...', temperature=0)

# TODO: Bind the tools to the LLM so that it knows their schemas and how to construct tool calls in JSON
llm_with_tools = llm.bind_tools(...)

# TODO: Create a ToolNode using our tools list
tool_node = ToolNode(...)

In [ ]:
# Test for Exercise 4.1
try:
    from langgraph.graph import StateGraph
    from langchain_openai import ChatOpenAI
    from langgraph.prebuilt import ToolNode
    assert isinstance(graph_builder, StateGraph), "graph_builder should be an instance of StateGraph"
    assert isinstance(llm, ChatOpenAI), "llm should be an instance of ChatOpenAI"
    # llm_with_tools should expose an invoke method
    assert hasattr(llm_with_tools, 'invoke'), "llm_with_tools should have an invoke method"
    assert isinstance(tool_node, ToolNode), "tool_node should be an instance of ToolNode"
    print("Initialization tests passed!")
except Exception as e:
    raise AssertionError(f"Initialization tests failed: {e}")

### Exercise 4.2 – Define the Chatbot Node

Define a chatbot function that takes the `state` as input and returns a dictionary with a single key `messages`.

When invoke the LLM, we provide it with all the `messages` in `state`.

In [94]:
from typing import Dict, List

# TODO: define chatbot node
def chatbot(state: ChatState) -> Dict[str, List]:
    """The main chatbot node. It invokes the LLM with the current messages."""
    return {'messages': [llm_with_tools.invoke(...)]}

In [ ]:
# Test for Exercise 4.2
try:
    assert callable(chatbot), 'chatbot should be callable'

    class _DummyLLM:
        def __init__(self):
            self.called = False
            self.last_messages = None
            self.response = 'dummy_response'
        def invoke(self, messages):
            self.called = True
            self.last_messages = messages
            return self.response

    target_callable = getattr(chatbot, '__wrapped__', chatbot)
    func_globals = getattr(target_callable, '__globals__', None)
    assert func_globals is not None, 'chatbot callable must expose globals for patching'

    had_original_llm = 'llm_with_tools' in func_globals
    original_llm = func_globals.get('llm_with_tools')
    dummy_llm = _DummyLLM()
    func_globals['llm_with_tools'] = dummy_llm

    try:
        test_state = {'messages': ['hello']}
        result = chatbot(test_state)
    finally:
        if had_original_llm:
            func_globals['llm_with_tools'] = original_llm
        else:
            func_globals.pop('llm_with_tools', None)

    assert dummy_llm.called, 'chatbot should call llm_with_tools.invoke'
    assert dummy_llm.last_messages == test_state['messages'], 'chatbot should pass the current state messages to the LLM'
    assert isinstance(result, dict) and 'messages' in result, "chatbot should return a dict with a 'messages' key"
    assert isinstance(result['messages'], list) and result['messages'], 'chatbot should return a non-empty messages list'
    assert result['messages'][0] == dummy_llm.response, 'chatbot should include the LLM response inside the messages list'
    print('Chatbot node tests passed!')
except Exception as e:
    raise AssertionError(f'Chatbot node tests failed: {e}')

### Exercise 4.3 – Define the Routing Function

Create a function `route_tools` that decides whether to call tools or terminate. 

- Step 1: Extract the last message from `state`.
- Step 2: If the last message contains tool calls (`tool_calls` attribute), return `'tools'`; otherwise return `END`.

In [ ]:
# TODO: define router function
def route_tools(state: ChatState) -> str:
    last_message = ...
    if getattr(last_message, ..., []):
        return ...
    return ...

In [ ]:
# Test for Exercise 4.3
try:
    assert callable(route_tools), 'route_tools should be callable'

    class _Msg:
        def __init__(self, tool_calls=None):
            self.tool_calls = tool_calls or []

    state_with_tools = {'messages': [_Msg(tool_calls=['call'])]}
    tool_branch = route_tools(state_with_tools)

    state_without_tools = {'messages': [_Msg(tool_calls=[])]}
    no_tool_branch = route_tools(state_without_tools)

    state_empty = {'messages': []}
    empty_branch = route_tools(state_empty)

    valid_tool_returns = {'tools', 'continue'}
    valid_end_returns = {'END', 'respond'}

    assert isinstance(tool_branch, str), 'route_tools should return a string when tools are requested'
    assert tool_branch in valid_tool_returns, "route_tools should return 'tools' or 'continue' when a tool call is present"
    for branch, label in [(no_tool_branch, 'no tool calls'), (empty_branch, 'no messages')]:
        assert isinstance(branch, str), f'route_tools should return a string when {label}'
        assert branch in valid_end_returns, "route_tools should return 'END' or 'respond' when no tool call is present"
    print('Routing function tests passed!')
except Exception as e:
    raise AssertionError(f'Routing function tests failed: {e}')

### Exercise 4.4 – Build the Complete Graph

Assemble the agent graph by adding nodes and edges, then compile the graph into a runnable agent. Use `add_node()` for your chatbot and tool nodes, `add_conditional_edges()` for routing logic, and `add_edge()` to connect nodes.


<img src="images/graph_create_1.png" width="800">

<img src="images/graph_create_2.png" width="800">

In [ ]:
graph_builder = StateGraph(ChatState)

# TODO: Add the chatbot node to the graph
graph_builder.add_node(...)

# TODO: Add the tool node to the graph
graph_builder.add_node(...)

# TODO: Add conditional edges from chatbot to either tools or END using the routing function
graph_builder.add_conditional_edges(...)

# TODO: Add edge from tools back to chatbot
graph_builder.add_edge(...)

# TODO: Add edge from START to chatbot
graph_builder.add_edge(...)

# TODO: Compile the graph into an agent
agent = graph_builder.compile(...)

In [ ]:
# Test for Exercise 4.4 
try:
    from langgraph.graph import START, END
    assert 'agent' in globals(), 'agent should be defined after building the graph'
    graph = agent.get_graph()
    node_names = set(graph.nodes)
    assert START in node_names and END in node_names, 'Compiled graph should include START and END nodes'
    assert {'chatbot', 'tools'}.issubset(node_names), "Graph should contain 'chatbot' and 'tools' nodes"

    edges = {(edge.source, edge.target, edge.data) for edge in graph.edges}
    assert (START, 'chatbot', None) in edges, "Graph should start at 'chatbot' from START"
    assert ('chatbot', 'tools', None) in edges, "Graph should route from 'chatbot' to 'tools' when tool calls are present"
    assert ('chatbot', END, 'END') in edges, "Graph should allow terminating at END from 'chatbot'"
    assert ('tools', 'chatbot', None) in edges, "Graph should loop from 'tools' back to 'chatbot'"
    print('Graph construction tests passed!')
except Exception as e:
    raise AssertionError(f'Graph construction tests failed: {e}')

### Exercise 4.5 – Visualize Your Agent

Use the graph's `draw_mermaid_png()` method to visualize the structure of your agent. This step is optional but helps you understand how nodes and edges connect.

In [ ]:
from IPython.display import Image, display


display(Image(agent.get_graph().draw_mermaid_png()))

---

## Part 5 – Testing Your Agent

Now that your agent graph is built, it's time to interact with it. Create a simple chat loop that greets the user, processes input until they type 'quit', and streams responses using `agent.stream()`.

### Exercise 5.1 – Create a Basic Chat Loop

Write an interactive loop that:
1. Greets the user and explains the agent's capabilities.
2. Reads user input in a loop and exits on `'quit'`, `'exit'` or `'q'`.
3. Creates the initial `state` with the user's message and streams the agent's responses via `agent.stream()`.
4. Uses the `pretty_print()` method on messages to display nicely formatted output.


Try out this chat loop with these three prompts: 

- **Prompt 1:** ” You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. What is the mechanism of action of remdesivir and what bioactivity data is available for this compound?”


- **Prompt 2:**  " You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. I'm investigating potential combination therapies for COVID-19. Can you help me understand the mechanisms of both hydroxychloroquine and azithromycin, then find recent literature discussing their combined use in COVID-19 treatment?”


- **Prompt 3:**  " You are an expert drug discovery researcher. Use your available tools to answer the user’s question as accurately as possible. Never fabricate or invent data. I'm working on Alzheimer's drug discovery and need to evaluate aducanumab. Please provide detailed drug information including its development status, then search for recent publications about its clinical trial outcomes and any controversies surrounding its approval."


In [ ]:
print('Welcome to the LangGraph ReAct demo with memory! Ask me a question.')
print('I can perform simple math, look up drug information, and search PubMed via LitSense.')
print('====================================================================')

while True:
    try:
        # Get input prompt from user
        user_input = input('User: ')
    except EOFError:
        break
    
    # If input contains one of three keywords: quit, exit, and q; exit the streaming process.
    if not user_input or user_input.lower() in {'quit', 'exit', 'q'}:
        print('Goodbye!')
        break
    
    # Initialize graph state with user's input
    state = {
        'messages': [
            {'role': 'user', 'content': user_input}
        ]
    }


    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    # Stream the answer from agent
    for event in agent.stream(state):
        for value in event.values():
            # The last message in the state is the AI response
            msg = value["messages"][-1]
            # Use the built-in pretty_print method for better formatting
            msg.pretty_print()

---

## Part 6: Adding Short‑term Memory

To demonstrate the lack of memory in the agent, please go back to Part 5, then perform these two prompts: 

- **Prompt 1:** My compound of interest is aspirin. Please remember this information.

- **Prompt 2:** What is my interesting compound?

You will see when invoking **prompt 2**, the agent can not come up with the correct answer - aspirin. This is because the GraphState is only maintained from the START node to the END node. After termination, all contexts from the previous run are lost. Therefore, we need a component called Agent Memory to maintain context across runs.

Memory allows your agent to remember previous parts of the conversation. LangGraph uses "checkpointers" to maintain state across interactions.

### Exercise 6.1 – Add Memory to Your Agent

Rebuild your agent with memory by creating a new `StateGraph` that uses the same `ChatState`. 

Initialize `checkpointer` with `InMemorySaver` to persist state across turns.

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

class ChatState(TypedDict):
    messages: Annotated[List[Dict], add_messages]

graph_builder = StateGraph(ChatState)

llm = ChatOpenAI(model='gpt-4o', temperature=0)
llm_with_tools = llm.bind_tools(TOOLS)

def chatbot(state: ChatState) -> Dict[str, List]:
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

tool_node = ToolNode(tools=TOOLS)

graph_builder.add_node('chatbot', chatbot)
graph_builder.add_node('tools', tool_node)
graph_builder.add_conditional_edges('chatbot', route_tools, {'tools': 'tools', END: END})
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')


# TODO: Add checkpointer to the agent
checkpointer = ...
agent = graph_builder.compile(checkpointer=...)

In [ ]:
# Test for Exercise 6.1
try:
    from langgraph.checkpoint.memory import InMemorySaver
    # Determine which variable holds the checkpointer
    cp = None
    for name in ['checkpointer', 'saver']:
        if name in globals():
            cp = globals()[name]
            break
    assert cp is not None and isinstance(cp, InMemorySaver), "A checkpointer instance of InMemorySaver should be created"
    assert 'agent' in globals(), "agent should be defined"
    # If agent exposes a checkpointer attribute, ensure it's InMemorySaver
    if hasattr(agent, 'checkpointer'):
        assert isinstance(agent.checkpointer, InMemorySaver), "agent.checkpointer should be an instance of InMemorySaver"
    print("Memory agent tests passed!")
except Exception as e:
    raise AssertionError(f"Memory agent tests failed: {e}")

### Exercise 6.2 – Create a Memory‑Enabled Chat Loop

Write a chat loop similar to Part 5, but supply a `config` dictionary to `agent.stream()`. In the config, there are two important parameters, including `thread_id` and `recursion_limit`.

<img src="images/short_term_mem.png" width="800">

In [ ]:
# TODO: Create config for memory
config = {
    'configurable': {
        '...': ...,        # Specify thread id
    },
    'recursion_limit': ...      # Specify recursion limit
}

In [ ]:
# Test for Exercise 6.2
try:
    assert isinstance(config, dict), "config should be a dictionary"
    assert 'configurable' in config, "config should contain 'configurable' key as first level key"
    assert 'recursion_limit' in config, "config should contain 'recursion_limit' key as first level key"
    assert isinstance(config['recursion_limit'], int) and config['recursion_limit'] > 0, "recursion_limit should be a positive integer"
    assert isinstance(config['configurable'], dict), "config['configurable'] should be a dictionary"
    assert 'thread_id' in config['configurable'], "config['configurable'] should contain 'thread_id'"
    assert isinstance(config['configurable']['thread_id'], str) and config['configurable']['thread_id'], "thread_id should be a non-empty string"
    print("Config creation tests passed!")
except Exception as e:
    raise AssertionError(f"Config creation tests failed: {e}")


In [ ]:
print('Welcome to the LangGraph ReAct demo with memory! Ask me a question.')
print('I can perform simple math, look up drug information, and search PubMed via LitSense.')
print('====================================================================')

while True:
    try:
        # Get input prompt from user
        user_input = input('User: ')
    except EOFError:
        break
    
    # If input contains one of three keywords: quit, exit, and q; exit the streaming process.
    if not user_input or user_input.lower() in {'quit', 'exit', 'q'}:
        print('Goodbye!')
        break

    # Initialize graph state with user's input
    state = {
        'messages': [
            {'role': 'user', 'content': user_input}
        ]
    }

    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    
    # TODO: Supply config for the agent stream process
    for event in agent.stream(state, config=...):
        for value in event.values():
            # The last message in the state is the AI response
            msg = value["messages"][-1]
            # Use the built-in pretty_print method for better formatting
            msg.pretty_print()

---

## Part 7 – Prebuilt Agents

LangGraph provides prebuilt agents that implement common architectures such as the ReAct pattern. These agents are quick to set up and let you focus on your tools and prompts rather than on graph wiring.

### Exercise 7.1 – Create a Prebuilt Agent

Use `create_react_agent` to instantiate a prebuilt ReAct agent. Provide your LLM, tools list, and a custom system prompt that instructs the agent how to behave.

#### References: [LangGraph Prebuilt Tutorial](https://langchain-ai.github.io/langgraph/agents/agents)

In [ ]:
# Create config for memory
config = {
    'configurable': {
        '...': ...,
    },
    'recursion_limit': ...
}

In [ ]:
# Test for Exercise 6.2 – Create a Memory‑Enabled Chat Loop Config
try:
    assert isinstance(config, dict), "config should be a dictionary"
    assert 'configurable' in config, "config should contain 'configurable' key as first level key"
    assert 'recursion_limit' in config, "config should contain 'recursion_limit' key as first level key"
    assert isinstance(config['recursion_limit'], int) and config['recursion_limit'] > 0, "recursion_limit should be a positive integer"
    assert isinstance(config['configurable'], dict), "config['configurable'] should be a dictionary"
    assert 'thread_id' in config['configurable'], "config['configurable'] should contain 'thread_id'"
    assert isinstance(config['configurable']['thread_id'], str) and config['configurable']['thread_id'], "thread_id should be a non-empty string"
    print("Config creation tests passed!")
except Exception as e:
    raise AssertionError(f"Config creation tests failed: {e}")


In [ ]:
from langchain.agents import create_agent

llm = ChatOpenAI(model='gpt-4o', temperature=0)

checkpointer = InMemorySaver()

agent = create_agent(
    model = llm,
    tools = TOOLS,
    system_prompt="You are a helpful assistant. Be concise and accurate.",
    checkpointer = checkpointer
)

In [ ]:
print('Welcome to the LangGraph ReAct demo with memory! Ask me a question.')
print('I can perform simple math, look up drug information, and search PubMed via LitSense.')
print('====================================================================')

while True:
    try:
        # Get input prompt from user
        user_input = input('User: ')
    except EOFError:
        break
    
    # If input contains one of three keywords: quit, exit, and q; exit the streaming process.
    if not user_input or user_input.lower() in {'quit', 'exit', 'q'}:
        print('Goodbye!')
        break

    # Initialize graph state with user's input
    state = {
        'messages': [
            {'role': 'user', 'content': user_input}
        ]
    }

    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    
    # Supply config for the agent stream process
    for event in agent.stream(state, config=config):
        for value in event.values():
            # The last message in the state is the AI response
            msg = value["messages"][-1]
            # Use the built-in pretty_print method for better formatting
            msg.pretty_print()

---

## Part 8 – Extension Exercises


### Exercise 8.1 – Create a system prompt using a prompt template

In LangGraph, you can provide a **system** or **instruction** message that sets the behaviour of the assistant before any user input is processed. LangGraph exposes this functionality via its `prompts` module. There you will find a few classes: `PromptTemplate` for formatting a single string, `ChatPromptTemplate` for building multi-message prompts, and `MessagesPlaceholder` for inserting a slot where conversation history will later be filled.

In this exercise, you will define a `ChatPromptTemplate` that begins with a system instruction. The system message should describe the assistant’s role (for example, instructing it to behave like a drug discovery researcher) and will be prepended to the user’s request before being passed to the agent.

#### API References: [Prompt Template](https://python.langchain.com/docs/concepts/prompt_templates/)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

#TODO: Implement ChatPromptTemplate here
prompt_template = ChatPromptTemplate.from_messages([
    ("...", "..."),
    ("user", "{request}")
])

#TODO: Invoke the prompt_template to construct prompt_values, which are prompts that will be sent to the system
prompt_values = prompt_template.invoke({'...':'What is the mechanism of action of remdesivir and what bioactivity data is available for this compound?'})

`prompt_values` is a dictionary, which can be invoke directly to the agent. Try out by simple invoke the agent with the `prompt_values`

In [ ]:
msgs = agent.invoke(prompt_values, config=config)
for msg in msgs['messages']:
    msg.pretty_print()

### Exercise 8.2 – Integrate the system prompt with a streaming loop

Building on the previous exercise, implement a multi-turn chat loop. Use a `while` loop to repeatedly accept user input, stream the agent’s response, and append both user and assistant messages to the conversation state.

To include your system instruction in every turn, apply the `ChatPromptTemplate` from exercise 8.1 when constructing the input for the agent. 

When using the streaming API you must supply a `state` dictionary that includes a `'messages'` key. The value of this key should be a list of message dictionaries (each with `role` and `content` fields) representing the conversation history. 

As you loop, update this list to maintain the context across turns.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

config = {
    "configurable": {
        "thread_id": "example_conversation",
        "recursion_limit": 25,
    }
}


prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert drug discovery researcher. Use your available tools to answer the user's question as accurately as possible. Never fabricate or invent data."),
    ("human", "{request}"),
])

print("Welcome to the LangGraph ReAct demo with memory! Ask me a question.")
print("I can perform simple math, look up drug information, and search PubMed via LitSense.")
print("====================================================================")

while True:
    try:
        user_input = input("User: ")
    except EOFError:
        break

    if not user_input or user_input.lower() in {"quit", "exit", "q"}:
        print("Goodbye!")
        break

    #TODO: Invoke prompt_template to construct prompt_values, then initilize state with prompt_values
    prompt_values = prompt_template.invoke({'...': ...})
    state = {"messages": prompt_values.messages}

    # Print the user's message first
    print(f"""================================== User's Message ==================================
{user_input}
    """)
    
    # Stream agent events
    for event in agent.stream(state, config=config):
        for value in event.values():
            msg = value["messages"][-1]
            msg.pretty_print()

### Exercise 9.1 - Structure Output Parser

Structured output parsers let you control the format of the language model’s responses. Instead of returning free‑form text, the model follows a schema (such as a Pydantic model) so that other components can reliably consume its output.

- **Why?** Downstream software often expects data in a specific JSON schema (e.g. key–value pairs).
- **How?** Define a Pydantic `BaseModel` to describe the fields you need, then use `with_structured_output(...)` to attach this schema to your LLM.

You can read more in the [LangGraph structured output guide](https://langchain-ai.github.io/langgraph/how-tos/react-agent-structured-output/#define-model-tools-and-graph-state).

The diagram below shows the high‑level structure of the system we’re about to build. One LLM is responsible for deciding when to call tools, and a second LLM is tasked with formatting the final response according to your schema.

<img src="images/react_structured_output.png" width="300">


First, we'll set up two language models:

1. **Tool‑calling LLM**: This model can invoke the tools you defined earlier (calculator, literature search, etc.).
2. **Structured output LLM**: This model will use a Pydantic schema to return the final answer as JSON with fields for drug names, justifications, and sources.

When we later ask a question such as *"Use LitSearch literature, give me two antiviral drugs that are potentially to treat COVID‑19"*, the agent will search PubMed via LitSense, find candidate antivirals, and then format its answer using this schema.



In [ ]:
# Step 1: create the tool-calling LLM
llm = ChatOpenAI(model='gpt-5', temperature=0)
llm_with_tools = llm.bind_tools(TOOLS)

# Step 2: define a Pydantic schema describing the structured output we want
from pydantic import BaseModel, Field

class DrugsInfo(BaseModel):
    drugs: str = Field(description='a list of drug names')
    justifications: str = Field(description='a list of justification for each drugs')
    sources: str = Field(description='a list of citation for each drugs')

# Step 3: create the structured output LLM
llm = ChatOpenAI(model='gpt-4o', temperature=0)
llm_with_structured_outputs = llm.with_structured_output(DrugsInfo)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from typing import Dict, List

# Define the chatbot node: this function invokes the tool‑calling LLM and appends the AI message to state
def chatbot(state: ChatState) -> Dict[str, List]:
    """The main chatbot node. It invokes the LLM with the current messages."""
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

# Define the respond node: this function uses the structured‑output LLM to format the final answer
def respond(state: ChatState):
    # Extract only the content of each message for the structured LLM
    response = llm_with_structured_outputs.invoke(
        [HumanMessage(content=state["messages"][i].content) for i in range(len(state["messages"]))]
    )
    # Wrap the structured JSON as an AIMessage and return it
    return {"messages": [AIMessage(content=response.model_dump_json())]}

# TODO: efine a  routing function to decide whether to keep calling tools or format a final response
def route_tools(state: ChatState):
    ...
    if ...:
        return ...
    return ...

<img src="images/graph_create_1.png" width="800">

<img src="images/graph_create_2.png" width="800">

In [ ]:
from langgraph.graph import StateGraph, END, START

# Build the state graph
graph_builder = StateGraph(ChatState)

# TODO: Add the nodes: chatbot, respond, and tools
graph_builder.add_node(...)
graph_builder.add_node(...)
graph_builder.add_node(...)

# TODO: Add the edges
graph_builder.add_edge(...) # from START to chatbot
graph_builder.add_conditional_edges(...) # from chatbot to respond or tools
graph_builder.add_edge(...) # from tools to chatbot
graph_builder.add_edge(...) # from respond to END

# Compile the workflow into an executable agent
agent = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception:
    print('Graph visualization not available in this environment.')

In [ ]:
# Ask the agent a question and pretty‑print the structured response
answer = agent.invoke({'messages': 'Use LitSearch literatures, give me 2 antiviral drugs that are potential to treat COVID-19'})
answer['messages'][-1].pretty_print()

---

## Key Takeaways

- **Nodes** are functions that operate on state.
- **Edges** define the flow between nodes.
- **State** carries data through the agent. Use reducer functions such as `add_messages` to control how the state is updated.
- **Tools** extend agent capabilities and are annotated with `@tool`.
- **Memory** allows agents to maintain context across invocations.
- **Prebuilt agents** provide quick solutions for common patterns but offer less control than a custom graph.

---
## Resources

- **LangChain Documentation** – https://docs.langchain.com/oss/python/langchain/overview
- **LangGraph Documentation** – https://docs.langchain.com/oss/python/langgraph/overview
- **ChEMBL Web Services** – https://chembl.gitbook.io/chembl-interface-documentation/web-services
- **OpenAI API** – https://platform.openai.com/docs/

These references can help you explore LangGraph and related libraries beyond the scope of this workshop.

---

**🎉 Congratulations! You've built your first AI agent with LangGraph!**

Feel free to modify and extend your agent. You can experiment with new tools, different LLMs, and additional nodes to create even more capable and personalised agents.